In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

from scipy.interpolate import interp1d

from ripser import ripser
from persim import plot_diagrams
from persim import wasserstein, wasserstein_matching
from persim import bottleneck, bottleneck_matching

In [ ]:
from data_cube import DataCube
from TDA_helper_fcns import sublevel_set_time_series_dist

In [ ]:
dc = DataCube(subjects="all",
              gestures="all",
              channels=["2", "4", "5", "6", "8"],
              data_grp="parsed")

dc.load_data()
dc.normalize_modalities()
dc.rms_smooth(100, 50)
dc.interpolate_modalities()

In [ ]:
test_sig = dc.data_set_interp["10"]["5_0_1"]

In [ ]:
# Setup the sliding window code
def getSlidingWindow(x, dim, Tau, dT):
    """
    x - The 1-D signal as a numpy array
    dim - window size i.e. dimension of output vectors/ embedding dimension
    Tau - skip between samples in a given window
    dT - The distance to slide between windows
    """
    N = len(x)
    NWindows = int(np.floor((N-dim*Tau)/dT)) # The number of windows
    if NWindows <= 0:
        print("Error: Tau too large for signal extent")
        return np.zeros((3, dim))
    X = np.zeros((NWindows, dim)) # Create a 2D array which will store all windows
    idx = np.arange(N)
    for i in range(NWindows):
        # Figure out the indices of the samples in this window
        idxx = dT*i + Tau*np.arange(dim) # index to sample on iteration i
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))+2
        if end >= len(x):
            X = X[0:i, :]
            break
        # Do spline interpolation to fill in this window, and place
        # it in the resulting array
        f = interp1d(idx[start:end+1], x[start:end+1], kind="cubic")
        X[i, :] = f(idxx)
    return X

In [ ]:
# Step 1: Setup the signal
t = test_sig[:, 0]
x = test_sig[:, 4]
N = t.size

In [ ]:
plt.plot(t,x)
plt.show()

In [ ]:
# Step 2: Do a sliding window embedding
dim = 30
Tau = 10
dT = 10
X = getSlidingWindow(x, dim, Tau, dT)
extent = Tau*dim

In [ ]:
# Step 3a: Do Rips Filtration for sls
sls = sublevel_set_time_series_dist(x) # LIMITING TO MODALITY 1
PDs = ripser(sls, maxdim=1,distance_matrix=True)['dgms']
I = PDs[1]

In [ ]:
# plot sublevel-set PD
# note that only 0-cycles exist
plot_diagrams(PDs)

In [ ]:
# Step 3b: Do Rips Filtration
PDs = ripser(X, maxdim=1)['dgms']
I = PDs[1]

In [ ]:
# plot sliding window embedding PD
plot_diagrams(PDs)

In [ ]:
# Step 4: Perform PCA down to 2D for visualization
pca = PCA(n_components = 2)
Y = pca.fit_transform(X)
eigs = pca.explained_variance_
print(pca.explained_variance_ratio_.sum())

In [ ]:
plt.scatter(Y[:,0],Y[:,1])
plt.show()

---

In [ ]:
def tally_votes(votes):
    """
    count up number of appearances of unique
    elements in a numpy array
    return element with most votes
    """
    unq, cnts = np.unique(votes, return_counts=True)
    d = dict(zip(cnts,unq))
    return int(d[max(d.keys())])

---

In [ ]:
mod = 2 # limiting to 1 modality for now
ts_lst = []
pd_lab = []
sbj_lab = []
for s, gdict in dc.data_set_interp.items():
    for g, a in gdict.items():
        ts_lst.append(a[:,mod])
        sbj_lab.append(s)
        pd_lab.append(int(g[0]))
        
pd_lab = np.array(pd_lab)

In [ ]:
dim = 30
Tau = 5
dT = 10

pd_lst = []
for i,ts in enumerate(ts_lst):
    # sliding window embedding
    X = getSlidingWindow(ts, dim, Tau, dT)
    # compute homology groups of embedding
    pd_lst.append(ripser(X, maxdim=1)['dgms'][1]) # H1 cycles

In [ ]:
# Wasserstein distance
N = len(pd_lab)
dw_mat = np.zeros(shape=(N,N))
for i in range(N):
    for j in range(N):
        if i <= j: continue # upper triangular only
        dw_mat[i,j] = wasserstein(pd_lst[i],pd_lst[j])

dw_mat = dw_mat + dw_mat.T

In [ ]:
k = 1
pred_lst = []
for i in range(N):
    kNN_idx = np.argsort(dw_mat[i,:])[1:k+1]
    votes = pd_lab[kNN_idx]
    pred_lst.append(tally_votes(votes))

In [ ]:
sum(np.array(pred_lst) == pd_lab) / pd_lab.size

In [ ]:
# bottleneck distance
N = len(pd_lab)
db_mat = np.zeros(shape=(N,N))
for i in range(N):
    for j in range(N):
        if i <= j: continue # upper triangular only
        db_mat[i,j] = bottleneck(pd_lst[i],pd_lst[j])

dw_mat = db_mat + db_mat.T

In [ ]:
k = 1
pred_lst = []
for i in range(N):
    kNN_idx = np.argsort(db_mat[i,:])[1:k+1]
    votes = pd_lab[kNN_idx]
    pred_lst.append(tally_votes(votes))

In [ ]:
sum(np.array(pred_lst) == pd_lab) / pd_lab.size

---

In [ ]:
def make_plots(ts,emb,PDs):
    """
    x - raw time series (1D numpy array)
    emb - sliding window embedding reduced to 2D (2D numpy array)
    PDs - persistence diagrams (H0 and H1)
    """
    plt.figure(figsize=(12,6))
    plt.subplot(131)
    plt.scatter(np.arange(ts.size),ts)
    
    plt.subplot(132)
    plt.scatter(emb[:,0],emb[:,1])
    
    plt.subplot(133)
    plot_diagrams(PDs)
    
    plt.show()

In [ ]:
dim = 30
Tau = 10
dT = 10

pca = PCA(n_components = 2)
pov_lst = []

for i,ts in enumerate(ts_lst):
    print(f"Subject {sbj_lab[i]}; gesture {gest_lab[i]}")
    # sliding window embedding
    X = getSlidingWindow(ts, dim, Tau, dT)
    # perform PCA on embedding
    Y = pca.fit_transform(X)
    pov = pca.explained_variance_ratio_.sum()
    pov_lst.append(pov)
    print(f"2 component PoV: {pov}")
    # compute homology groups of embedding
    PDs = ripser(X, maxdim=1)['dgms']
    
    # generate plots
    make_plots(ts,Y,PDs)